In [3]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import Dense, Flatten, Reshape, InputLayer

In [4]:
(X_train, _), (X_test, _) = mnist.load_data()

In [5]:
X_train = X_train.astype('float32') / 255

In [6]:
X_train = tf.data.Dataset.from_tensor_slices(X_train).shuffle(60000).batch(256)

In [7]:
# Generator
generator = Sequential([
    InputLayer(input_shape=(100,)),
    Dense(units=500, input_dim=100, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)),
    Dense(units=500, input_dim=100, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)),
    Dense(units=784, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)),
    Reshape(target_shape=(28, 28))
])

generator.summary()

c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 500)            │        50,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 784)            │       392,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,784 (2.65 MB)

 Trainable params: 693,784 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Discriminator
discriminator = Sequential([
    InputLayer(input_shape=(28, 28)),
    Flatten(),
    Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)),
    Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)),
    Dense(units=1, activation='sigmoid', kernel_regularizer=L1L2(1e-5, 1e-5))
])

discriminator.summary()

c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 500)            │       392,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,501 (2.45 MB)

 Trainable params: 643,501 (2.45 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [10]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [11]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [12]:
generetor_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [13]:
epochs = 100
noise_dim = 100

In [14]:
def train_step(images):
    noise = tf.random.normal([256, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generetor_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
for epoch in range(epochs):
    for image_batch in X_train:
        gen_loss, disc_loss = train_step(image_batch)

    print(f'Epoch {epoch + 1}, gen loss: {gen_loss}, disc loss: {disc_loss}')

Epoch 1, gen loss: 4.356014251708984, disc loss: 0.10061020404100418
Epoch 2, gen loss: 5.910799980163574, disc loss: 0.02332092449069023
Epoch 3, gen loss: 6.262994766235352, disc loss: 0.020443661138415337
Epoch 4, gen loss: 6.802391529083252, disc loss: 0.004721895791590214
Epoch 5, gen loss: 5.468331336975098, disc loss: 0.022350860759615898
Epoch 6, gen loss: 3.124769926071167, disc loss: 0.0668540894985199
Epoch 7, gen loss: 5.082282066345215, disc loss: 0.01170128770172596
Epoch 8, gen loss: 5.82880163192749, disc loss: 0.01589362882077694
Epoch 9, gen loss: 6.487020015716553, disc loss: 0.011139795184135437


In [ ]:
samples = np.random.normal(size=(20, 100))
predictions = generator(samples)
for i in range(predictions.shape[0]):
    plt.imshow(predictions[i, :], cmap='gray')
    plt.axis('off')